In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

train = pd.read_csv("4th_week/data/application_train.csv")
test = pd.read_csv("4th_week/data/application_test.csv")

In [3]:
TRAIN_FLAG_SUM = train[['FLAG_DOCUMENT_2',
                        'FLAG_DOCUMENT_3',
                        'FLAG_DOCUMENT_4',
                        'FLAG_DOCUMENT_5',
                        'FLAG_DOCUMENT_6',
                        'FLAG_DOCUMENT_7',
                        'FLAG_DOCUMENT_8',
                        'FLAG_DOCUMENT_9',
                        'FLAG_DOCUMENT_10',
                        'FLAG_DOCUMENT_11',
                        'FLAG_DOCUMENT_12',
                        'FLAG_DOCUMENT_13',
                        'FLAG_DOCUMENT_14',
                        'FLAG_DOCUMENT_15',
                        'FLAG_DOCUMENT_16',
                        'FLAG_DOCUMENT_17',
                        'FLAG_DOCUMENT_18',
                        'FLAG_DOCUMENT_19',
                        'FLAG_DOCUMENT_20',
                        'FLAG_DOCUMENT_21']]
train['FLAG_DOCUMENT_SUM'] = TRAIN_FLAG_SUM.sum(axis=1)

In [4]:
TRAIN_PHONE_SUM = train[[
    'FLAG_MOBIL',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'FLAG_CONT_MOBILE',
    'FLAG_PHONE']]
train['PHONE_SUM'] = TRAIN_PHONE_SUM.sum(axis=1)

In [5]:
missing = train.isnull().sum()
missing = missing[missing >0]
missing.sort_values(inplace=True)
# missing.plot.bar()

In [6]:
# missing

In [7]:
train = train.drop(columns=['APARTMENTS_MEDI',
                            'BASEMENTAREA_MEDI',
                            'YEARS_BEGINEXPLUATATION_MEDI',
                            'YEARS_BUILD_MEDI',
                            'COMMONAREA_MEDI',
                            'ELEVATORS_MEDI',
                            'ENTRANCES_MEDI',
                            'FLOORSMAX_MEDI',
                            'FLOORSMIN_MEDI',
                            'LANDAREA_MEDI',
                            'LIVINGAPARTMENTS_MEDI',
                            'LIVINGAREA_MEDI',
                            'NONLIVINGAPARTMENTS_MEDI',
                            'NONLIVINGAREA_MEDI',
                            'APARTMENTS_MODE',
                            'BASEMENTAREA_MODE',
                            'YEARS_BEGINEXPLUATATION_MODE',
                            'YEARS_BUILD_MODE',
                            'COMMONAREA_MODE',
                            'ELEVATORS_MODE',
                            'ENTRANCES_MODE',
                            'FLOORSMAX_MODE',
                            'FLOORSMIN_MODE',
                            'LANDAREA_MODE',
                            'LIVINGAPARTMENTS_MODE',
                            'LIVINGAREA_MODE',
                            'NONLIVINGAPARTMENTS_MODE',
                            'NONLIVINGAREA_MODE',
                            'FONDKAPREMONT_MODE',
                            'HOUSETYPE_MODE',
                            'TOTALAREA_MODE',
                            'WALLSMATERIAL_MODE',
                            'EMERGENCYSTATE_MODE'
                            ,
                            'FLAG_DOCUMENT_2',
                            'FLAG_DOCUMENT_3',
                            'FLAG_DOCUMENT_4',
                            'FLAG_DOCUMENT_5',
                            'FLAG_DOCUMENT_6',
                            'FLAG_DOCUMENT_7',
                            'FLAG_DOCUMENT_8',
                            'FLAG_DOCUMENT_9',
                            'FLAG_DOCUMENT_10',
                            'FLAG_DOCUMENT_11',
                            'FLAG_DOCUMENT_12',
                            'FLAG_DOCUMENT_13',
                            'FLAG_DOCUMENT_14',
                            'FLAG_DOCUMENT_15',
                            'FLAG_DOCUMENT_16',
                            'FLAG_DOCUMENT_17',
                            'FLAG_DOCUMENT_18',
                            'FLAG_DOCUMENT_19',
                            'FLAG_DOCUMENT_20',
                            'FLAG_DOCUMENT_21',
                            'FLAG_MOBIL',
                            'FLAG_EMP_PHONE',
                            'FLAG_WORK_PHONE',
                            'FLAG_CONT_MOBILE',
                            'FLAG_PHONE'
                            ])

In [8]:
log_transform('AMT_CREDIT')
log_transform('AMT_ANNUITY')
log_transform('AMT_GOODS_PRICE')
log_transform('AMT_INCOME_TOTAL')

train['NAME_CONTRACT_TYPE'] = train['NAME_CONTRACT_TYPE'].apply(lambda x: 1 if x == 'Cash loans' else 0)
train['FLAG_OWN_CAR'] = train['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'y' else 0)
train['AMT_INCOME_TOTAL'] = train['AMT_INCOME_TOTAL'].apply(lambda x: 1 if x > 13.3 else 0)

def func_NAME_EDUCATION_TYPE(x):
    if x == 'Higher education':
        return 1
    elif x == 'Academic degree':
        return 1
    else:
        return 0
train['NAME_EDUCATION_TYPE'] = train['NAME_EDUCATION_TYPE'].apply(func_NAME_EDUCATION_TYPE)

def func_NAME_HOUSING_TYPE(x):
    if x == 'Maternity leave':
        return 1
    elif x == 'Unemployede':
        return 1
    else:
        return 0
train['NAME_HOUSING_TYPE'] = train['NAME_HOUSING_TYPE'].apply(func_NAME_HOUSING_TYPE)

train['REGION_POPULATION_RELATIVE'] = train['REGION_POPULATION_RELATIVE'].apply(lambda x: 1 if x >= 0.02 else 0)
train['OWN_CAR_AGE'] = train['OWN_CAR_AGE'].apply(lambda x: 1 if x <= 10 else 0)


train['ORGANIZATION_spmean'] = train[['ORGANIZATION_TYPE', 'TARGET']].groupby(['ORGANIZATION_TYPE']).mean()['TARGET']
train['ORGANIZATION_TYPE'] = train['ORGANIZATION_spmean'].apply(lambda x: 1 if x > 0.08 else 0)


In [8]:
numerical = [f for f in train.columns if train.dtypes[f] != 'object']
numerical.remove('TARGET')
categorical = [f for f in train.columns if train.dtypes[f] == 'object']

In [9]:
# f = pd.melt(train, value_vars=numerical)
# g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False)
# g = g.map(sns.distplot, "value")

In [10]:
for c in categorical:
    train[c] = train[c].astype('category')
    if train[c].isnull().any():
        train[c] = train[c].cat.add_categories(['MISSING'])
        train[c] = train[c].fillna('MISSING')

In [11]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
# f = pd.melt(train, id_vars=['TARGET'], value_vars=categorical)
# g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
# g = g.map(boxplot, "value", "TARGET")

In [12]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    # ordering['spmean'] = frame[[feature, 'TARGET']].groupby(feature).mean()['TARGET']
    # ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()

    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o

cate_encoded = []
for q in categorical:
    encode(train, q)
    encode(test, q)
    cate_encoded.append(q+'_E')
# print(cate_encoded)

In [13]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['TARGET'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')

features = numerical + cate_encoded
# spearman(train, features)

In [14]:
def pairplot(x, y, **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame1({'time': x, 'val': y})
    ts = ts.groupby('time').mean()
    ts.plot(ax=ax)
    plt.xticks(rotation=90)

# f = pd.melt(train, id_vars=['TARGET'], value_vars=numerical+cate_encoded)
# g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
# g = g.map(pairplot, "value", "TARGET")

In [15]:
def error(actual, predicted):
    actual = np.log(actual)
    predicted = np.log(predicted)
    return np.sqrt(np.sum(np.square(actual-predicted))/len(actual))

def log_transform(feature):
    train[feature] = np.log1p(train[feature].values)

def quadratic(feature):
    train[feature+'2'] = train[feature]**2

In [17]:
split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42, )
for train_index, test_index in split.split(train, train["NAME_INCOME_TYPE"]):
    train_set = train.loc[train_index]
    test_set = train.loc[test_index]

In [18]:
features = numerical + cate_encoded
# X_train = train[features].fillna(0.).values
# y_train = train['TARGET'].values
# X_test = test[features].fillna(0.).values


X_train = train_set[features].fillna(0.).values
y_train = train_set['TARGET'].values
X_test = test_set[features].fillna(0.).values
y_test = test_set['TARGET'].values

In [24]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_sample(X_train, y_train)


In [25]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


logreg = LogisticRegression()
logreg.fit(X_resampled, y_resampled)
y_pred = logreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

0.6000202516589057

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
# rf.fit(X_resampled, y_resampled)
# y_pred_rf = rf.predict_proba(X_test)[:, 1]
# roc_auc_score(y_test, y_pred_rf)


param_grid = {
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_resampled, y_resampled)
CV_rfc.best_params_

# y_pred_rf = rf.predict_proba(X_test)[:, 1]
# roc_auc_score(y_test, y_pred_rf)

In [22]:
# my_submission = pd.DataFrame({'SK_ID_CURR': test.SK_ID_CURR, 'TARGET': y_pred})
# my_submission.to_csv('submission.csv', index=False)